In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D,Dropout,Flatten,Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import numpy as np
import os

In [18]:
DIRECTORY = r"D:\Facemask Detection\data"
CATEGORIES = ["with_mask", "without_mask"]

data = []
labels = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category)
    for img in os.listdir(path):
        img_path = os.path.join(path, img)
        image = load_img(img_path, target_size=(224,224))
        image = img_to_array(image)
        image = preprocess_input(image)
        data.append(image)
        labels.append(category)

In [19]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [20]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

X_train,X_test,y_train,y_test = train_test_split(data, labels,test_size=0.20, stratify=labels, random_state=42)

In [22]:
baseModel = MobileNetV2(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))

# the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128,input_shape= (224,224)activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

model = Model(inputs=baseModel.input, outputs=headModel)

In [23]:
for layer in baseModel.layers:
    layer.trainable = False

In [21]:
aug = ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,fill_mode="nearest")

In [24]:
model.compile(loss="binary_crossentropy", optimizer='adam',metrics=["accuracy"])
model.fit(aug.flow(X_train, y_train, batch_size=32),steps_per_epoch=len(X_train)//32,validation_data=(X_test, y_test),validation_steps=len(X_test)//32,epochs=20)

Epoch 1/20
188/188 [==============================] - ETA: 0s - loss: 0.0984 - accuracy: 0.9651WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 47 batches). You may need to use the repeat() function when building your dataset.


188/188 [==============================] - 424s 2s/step - loss: 0.0984 - accuracy: 0.9651 - val_loss: 0.0521 - val_accuracy: 0.9828
Epoch 2/20
188/188 [==============================] - 348s 2s/step - loss: 0.0514 - accuracy: 0.9825
Epoch 3/20
188/188 [==============================] - 387s 2s/step - loss: 0.0488 - accuracy: 0.9840
Epoch 4/20
188/188 [==============================] - 339s 2s/step - loss: 0.0398 - accuracy: 0.9869
Epoch 5/20
188/188 [==============================] - 335s 2s/step - loss: 0.0335 - accuracy: 0.9889
Epoch 6/20
188/188 [==============================] - 332s 2s/step - loss: 0.0345 - accuracy: 0.9884
Epoch 7/20
188/188 [==============================] - 340s 2s/step - loss: 0.0239 - accuracy: 0.9917
Epoch 8/20
188/188 [==============================] - 340s 2s/step - loss: 0.0276 - accuracy: 0.9894
Epoch 9/20
188/188 [==============================] - 201s 1s/step - loss: 0.0249 - accuracy: 0.9910
Epoch 10/20
188/188 [==============================] - 216s 

In [25]:
predIdxs = model.predict(X_test, batch_size=32)
predIdxs = np.argmax(predIdxs, axis=1)

print(classification_report(y_test.argmax(axis=1), predIdxs,target_names=lb.classes_))

48/48 [==============================] - 38s 775ms/step
              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       745
without_mask       0.99      0.99      0.99       766

    accuracy                           0.99      1511
   macro avg       0.99      0.99      0.99      1511
weighted avg       0.99      0.99      0.99      1511



In [26]:
model.save("mask_detector.model", save_format="h5")

C:\Users\8mpra\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
